In [1]:
import pandas as pd

In [58]:
rc = pd.read_excel('../data/orion_rc.xlsx', skiprows=11)
rc = rc.dropna(axis=1, how='all').iloc[:,:3]#.set_index('Date')
rc.columns = ['Date', 'max_off', 'max_ave_off']
# rc['max_off'] = pd.to_timedelta(rc['max_off'])
# rc['max_ave_off'] = pd.to_numeric(rc['max_ave_off'], errors='coerce')
rc = rc.set_index('Date')

In [62]:
# convert h:m:s to seconds
rc['max_off'] = pd.to_datetime(rc['max_off'], format='%H:%M:%S')
rc['max_ave_off'] = pd.to_datetime(rc['max_ave_off'], format='%H:%M:%S')

In [77]:
rc.dtypes

max_off        datetime64[ns]
max_ave_off    datetime64[ns]
dtype: object

In [76]:
rc.assign(blah = lambda x: x.max_off.dt.timestamp())

AttributeError: 'DatetimeProperties' object has no attribute 'timestamp'